<a href="https://colab.research.google.com/github/qwert-top/NMM-recognition-with-OpenPose/blob/main/%E5%88%86%E6%9E%90%E6%9C%AC%E7%95%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


平叙文             |frame  |鼻先      |眉上げ   |首振り
-------------------|------------------|---------------|--------------|----------
私は佐藤です       | 80-120 |該当部分に|NULL|
あなたは水泳が好きです | 100,130    |11.66,22.90|176.36,177.48|8.57
あなたはハルミです | 145-189 |-6.02,6.32|160.55,163.13|1.11
あなたはこれを食べます | 169-219|10.68,15.27|162.98,167.21| 1.25
私と姉はハワイに行きます|
私はサッカーが好きです|73-100|3.17,5.46|170.96,173.66|9.63
私はテニスを練習します|95-120|12.75,20.72|165.80,166.99|3.83
あなたはコーヒーを飲みます|110-126|38.65,64.06|178.61,181.25|2.89
あなたはジュースを飲みますか|該当部分に|NULLがある

Yes/No疑問文           | 該当部分のframe  |鼻先      |眉上げ   |首振り
----------------|------------------|----------|---------|----------
明日学校に行きますか？|80-121|42.71,110.94|208.48,220.18| 2.83
あなたはアイスが好きですか？| 104-133| 31.98,71.14| 190.61,200.95|  2.13
あなたはこれを食べますか？| 195-250| 93.85,115.62| 194.88,204.86| 1.66
あなたはコーラを飲みますか？|122-151|該当部分に|NULL
あなたはハルミですか？| 120-169| 74.26,116.61| 179.74,197.29|2.84
あなたはバナナが好きですか？| 93-141| 51.50,95.75| 199.70,209.99|13.76
あなたはバナナを食べますか？| 91-119| 68.82,90.60|196.52,202.86|3.39
あなたは野球が好きですか？| 100-131|60.06-80.56|196.98-204.92|3.33
これは本ですか？| 67-91|80.08-89.21|201.38-204.22|4.14
たかしはコーヒーを飲みますか？|128-155|29.6-69.05|202.43-207.95|3.12
VVという表現方法を知っていますか？|176-203|27.74,74.52|215.57,224.97|24.34
私は寿司を食べますか？|85-104|85.05,89.36|203.76,207.40|3.59

WH疑問文           | 該当部分のframe  |鼻先      |眉上げ   |首振り
----------------|------------------|----------|---------|----------
最寄りの駅はどこですか？|90-133|-38.33,-14.29|190.40,196.38|31.73
あなたの出身地はどこですか？|145-223|25.39,74.29|185.76,201.72|33.63
あなたの好きなスポーツは何ですか？|127-165|-0.55,37.81|198.74,208.88|21.87
佐藤さんが図書館で本を読んだのはいつですか？|150-182|5.80,46.46|201.53,207.56|21.49
昨日佐藤さんが本を読んだのはどこですか？|150-180|-33.94,-14.39|190.67,204.99|12.01
昨日図書館で本を読んだのは誰ですか？|
あなたの誕生日はいつですか？
名前は何ですか？
あの女性は誰ですか？| 129-156
昨日図書館で佐藤さんが読んだ本は何ですか？|
佐藤さんはどこですか？

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import median_filter
import copy

data =  [pd.read_csv('/content/drive/MyDrive/proB/あなたはバナナが好きですか？/face_out.csv')]

text = ['あなたはバナナが好きですか？']

category = [[0,0,0]]


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/proB/あなたはバナナが好きですか？/face_out.csv'

In [ ]:
#位置補正
corrected_data = copy.deepcopy(data)
for n in range(len(data)):
  origin_x = corrected_data[n]['30_x'][0]
  origin_y = corrected_data[n]['30_y'][0]
  for part in range(70):
    col_x = str(part) + '_x'
    col_y = str(part) + '_y'
    for frame in range(len(data[n])):
      corrected_data[n][col_x][frame] = corrected_data[n][col_x][frame] - origin_x
      corrected_data[n][col_y][frame] = corrected_data[n][col_y][frame] - origin_y

In [ ]:
#スケール変換
converted_data = copy.deepcopy(corrected_data)
for n in range(len(data)):
  original_scale_x = converted_data[n]['27_x'][0] - converted_data[n]['0_x'][0]
  scale_conversion_x = 140 / original_scale_x #顔の横幅を140に固定
  original_scale_y = converted_data[n]['8_y'][0]- converted_data[n]['27_y'][0]
  scale_conversion_y =  210 / original_scale_y #顔の縦幅を210に固定

  for part in range(70):
    col_x = str(part) + '_x'
    col_y = str(part) + '_y'
    for frame in range(len(data[n])):
      converted_data[n][col_x][frame] = converted_data[n][col_x][frame] * scale_conversion_x
      converted_data[n][col_y][frame] = converted_data[n][col_y][frame] * scale_conversion_y

In [ ]:
start = 150
end = 180
print(str(start)+"-"+str(end))

In [ ]:
#眉毛と黒目の距離、最初との差
#眉上げが検出できるはず

def eyeblow_detector(data,th,num,start=None,end=None):
  fig, ax = plt.subplots()
  for n in range(len(data)):
    #start = data[n]['60_y'][0] - data[n]['24_y'][0]
    diff = data[n]['60_y'] - data[n]['24_y']
    result = diff
    ax.plot(result)
    if start != None:
      plt.xlim(start,end)

    check = (result > th)
    if check.sum() > num:
      print(check.sum())
      print(text[n] , "has 眉上げ")
      category[n][1] = 1
  plt.show()

eyeblow_detector(converted_data, 20, 30,150,180)
eyeblow_detector(converted_data, 20, 30)

In [ ]:
#鼻先のy座標、最初の座標との差
#顎引きが検出できるはず

def chin_detector(data,th,num,start=None,end=None): #th=顎引き検出のしきい値
  fig, ax = plt.subplots()
  ax.invert_yaxis()
  for n in range(len(data)):
    #start = data[n]['30_y'][0]
    result = data[n]['30_y']
    ax.plot(result)
    if start != None:
      plt.xlim(start, end)
    check = (result > th)
    if check.sum() > num:
      print(check.sum())
      print(text[n] , "has 顎引き")
      category[n][0] = 1
  plt.show()

chin_detector(converted_data,80, 1, 73,100)
chin_detector(converted_data,80, 1)

In [ ]:
#鼻先のy座標、最初の座標との差
#顎引きが検出できるはず

def chin_detector_median(data,th,num): #th=顎引き検出のしきい値
  fig, ax = plt.subplots()
  ax.invert_yaxis()
  for n in range(len(data)):
    #start = data[n]['30_y'][0]
    result = data[n]['30_y']
    ax.plot(median_filter(result, size=5, cval=0, mode='constant'))
    if start != None:
      plt.xlim(start, end)
    max_y = -10000
    min_y = 10000
    for i, (x, y) in enumerate(zip(range(len(median_filter(result, size=5, cval=0, mode='constant'))),median_filter(result, size=5, cval=0, mode='constant'))):
      if x >= start and x <= end:
        if y > max_y:
          max_x = x
          max_y = y
        if y < min_y:
          min_x = x
          min_y = y
    ax.annotate(f'Max: {max_y:.2f}', xy=(max_x, max_y), textcoords="offset points", xytext=(0, 5), ha='center')
    ax.annotate(f'Min: {min_y:.2f}', xy=(min_x, min_y), textcoords="offset points", xytext=(0, -15), ha='center')
    print(str(f'{min_y:.2f}')+','+str(f'{max_y:.2f}'))

    check = (result > th)
    if check.sum() > num:
      print(check.sum())
      print(text[n] , "has 顎引き")
      category[n][0] = 1
  plt.show()

chin_detector_median(converted_data,80, 1)

In [ ]:
#眉毛と黒目の距離、最初との差
#眉上げが検出できるはず

def eyeblow_detector_median(data,th,num):
  fig, ax = plt.subplots()
  for n in range(len(data)):
    #start = data[n]['60_y'][0] - data[n]['24_y'][0]
    diff = data[n]['60_y'] - data[n]['24_y']
    result = diff
    ax.plot(median_filter(result, size=5, cval=0, mode='constant'))
    if start != None:
      plt.xlim(start,end)
    max_y = -10000
    min_y = 10000
    for i, (x, y) in enumerate(zip(range(len(median_filter(result, size=5, cval=0, mode='constant'))),median_filter(result, size=5, cval=0, mode='constant'))):
      if x >= start and x <= end:
        if y > max_y:
          max_x = x
          max_y = y
        if y < min_y:
          min_x = x
          min_y = y
    ax.annotate(f'Max: {max_y:.2f}', xy=(max_x, max_y), textcoords="offset points", xytext=(0, 5), ha='center')
    ax.annotate(f'Min: {min_y:.2f}', xy=(min_x, min_y), textcoords="offset points", xytext=(0, -15), ha='center')
    print(str(f'{min_y:.2f}')+','+str(f'{max_y:.2f}'))


    check = (result > th)
    if check.sum() > num:
      print(check.sum())
      print(text[n] , "has 眉上げ")
      category[n][1] = 1
  plt.show()

eyeblow_detector_median(converted_data, 20, 30)

In [ ]:
#首振りを検出するために、一つ前のフレームとの座標の差を求める
#絶対値
def frame_diff_abs(data):
  ans = []
  for frame in range(1, len(data)):
    ans.append(abs(data['30_x'][frame] - data['30_x'][frame-1]))
  return ans

def filtered_head_shaking_detector(data, th, num):
  fig, ax = plt.subplots()
  for n in range(len(data)):
    ans = frame_diff_abs(data[n])
    ax.plot(median_filter(ans, size=5, cval=0, mode='constant'))
    if start != None:
      plt.xlim(start,end)
    max_y = -10000
    min_y = 10000
    for i, (x, y) in enumerate(zip(range(len(median_filter(ans, size=5, cval=0, mode='constant'))),median_filter(ans, size=5, cval=0, mode='constant'))):
      if x >= start and x <= end:
        if y > max_y:
          max_x = x
          max_y = y
        if y < min_y:
          min_x = x
          min_y = y
    ax.annotate(f'Max: {max_y:.2f}', xy=(max_x, max_y), textcoords="offset points", xytext=(0, 5), ha='center')
    ax.annotate(f'Min: {min_y:.2f}', xy=(min_x, min_y), textcoords="offset points", xytext=(0, -15), ha='center')
    print(f'{min_y:.2f}',f'{max_y:.2f}')

    check = sum([a > th for a in ans])
    if check > num:
      print(check)
      print(text[n] , "has 首振り")
      category[n][2] = 1
  plt.show()

filtered_head_shaking_detector(converted_data, 20, 35)

In [ ]:

#右目と左目をつなぐ線と直交して27を通る線より30が右にあるか左にあるか
#39と42

def line_equation(data):
    fig, ax = plt.subplots()
    ax.invert_yaxis()
    for n in range(len(data)):
      x1 = copy.deepcopy(data[n]['39_x'])
      x2 = copy.deepcopy(data[n]['42_x'])
      y1 = copy.deepcopy(data[n]['39_y'])
      y2 = copy.deepcopy(data[n]['42_y'])

      m1 = (y2-y1)/(x2-x1)
      center_x = copy.deepcopy(data[n]['0_x'])

      for frame in range(len(data[n])):
        if m1[frame] == 0:
          center_x[frame] = data[n]['27_x'][frame]
        else:
          slope = -1 / m1[frame]
          x = data[n]['27_x'][frame]
          y = data[n]['27_y'][frame]
          intercept = y - slope * x
          center_x[frame] = (data[n]['30_y'][frame] - intercept)/slope
      ax.plot(data[n]['30_x']-center_x, data[n]['Unnamed: 0'])
    plt.show()

line_equation(converted_data)



In [ ]:
#首振りを検出するために、一つ前のフレームとの座標の差を求める
#絶対値
def frame_diff_abs(data):
  ans = []
  for frame in range(1, len(data)):
    ans.append(abs(data['30_x'][frame] - data['30_x'][frame-1]))
  return ans

def abs_head_shaking_detector(data, th, num):
  fig, ax = plt.subplots()
  for n in range(len(data)):
    ans = frame_diff_abs(data[n])
    ax.plot(ans)
    check = sum([a > th for a in ans])
    if check > num:
      print(check)
      print(text[n] , "has 首振り")
      category[n][2] = 1
  plt.show()

abs_head_shaking_detector(converted_data, 20, 35)

In [ ]:

#右目と左目をつなぐ線と直交して27を通る線より30が右にあるか左にあるか
#39と42

def line_equation(data):
    fig, ax = plt.subplots()
    ax.invert_yaxis()
    for n in range(len(data)):
      x1 = copy.deepcopy(data[n]['39_x'])
      x2 = copy.deepcopy(data[n]['42_x'])
      y1 = copy.deepcopy(data[n]['39_y'])
      y2 = copy.deepcopy(data[n]['42_y'])

      m1 = (y2-y1)/(x2-x1)
      center_x = copy.deepcopy(data[n]['0_x'])
      which = [0]*len(data[n])

      for frame in range(len(data[n])):
        if m1[frame] == 0:
          center_x[frame] = data[n]['27_x'][frame]
        else:
          slope = -1 / m1[frame]
          x = data[n]['27_x'][frame]
          y = data[n]['27_y'][frame]
          intercept = y - slope * x
          center_x[frame] = (data[n]['30_y'][frame] - intercept)/slope
          if data[n]['30_x'][frame]-center_x[frame] > 0:
            which[frame] = 5
          else:
            which[frame] = -5

      ax.plot(data[n]['30_x']-center_x)
      ax.plot(which)
    plt.show()

line_equation(converted_data)



In [ ]:

from scipy.signal import butter, filtfilt

fig, ax = plt.subplots()

# ローパスフィルタ
def butter_lowpass_filter(data, cutoff_frequency, sampling_rate, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

for n in range(len(converted_data)):
   ans = frame_diff_abs(converted_data[n])

# フィルタリング
cutoff_frequency = 5  # ローパスフィルタのカットオフ周波数
filtered_data = butter_lowpass_filter(ans, cutoff_frequency, 1 / 0.01)

# 微分
derivative_data = np.gradient(filtered_data)

# 閾値の設定
threshold = 0.5

# 小刻みな首振りの検出
swings = np.where(np.abs(derivative_data) > threshold)[0]

# 結果のプロット
plt.plot(ans, label='Original Data')
plt.plot(filtered_data, label='Filtered Data')
plt.plot(derivative_data, label='Derivative')
plt.legend()
plt.show()


In [ ]:

def butter_highpass_filter(data, cutoff_frequency, sampling_rate, order=4):
    nyquist = 0.5 * sampling_rate
    high = cutoff_frequency / nyquist
    b, a = butter(order, high, btype='high', analog=False)
    y = filtfilt(b, a, data)
    return y


# ハイパスフィルタ
cutoff_frequency = 2  # ハイパスフィルタのカットオフ周波数
sampling_rate = 1 / 0.01  # サンプリングレート
order = 4  # 次数

filtered_data = butter_highpass_filter(converted_data[0]['30_x'], cutoff_frequency, sampling_rate)
# 微分
derivative_data = np.gradient(filtered_data)

# 閾値の設定
threshold = 0.5
# 結果のプロット
plt.plot(converted_data[0]['30_x'], label='Original Data')
plt.plot(filtered_data, label='Filtered Data (Highpass)')
plt.plot(derivative_data, label='Derivative')
plt.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots()
for n in range(len(converted_data)):
   ans = frame_diff_abs(converted_data[n])
   ax.plot(ans)
plt.show()
